<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy
import tensorflow as tf
import tarfile
import os
import sys
import warnings

warnings.filterwarnings('ignore')